In [1]:
import random
import numpy as np
import itertools
import copy

## Define a linear function for f

In [45]:
n = 10
t = 300
x = np.random.uniform(0,1, n)
a = np.random.uniform(-1,1, n)

In [46]:
x

array([0.39258359, 0.17741823, 0.59250329, 0.54281811, 0.33530203,
       0.62242382, 0.31806745, 0.12839349, 0.88517413, 0.24545588])

In [47]:
a

array([ 0.22190737, -0.70470618, -0.28948519,  0.35585917,  0.44527886,
       -0.74862312, -0.30095507, -0.87786712, -0.1669726 , -0.95652113])

In [48]:
def f_linear(x):   
    return np.dot(a,x.T)

f_linear(x)

-0.9239426147152634

## Approximation for Multi-Linear Extension

In [49]:
x_sample = 0

def F_unfixed(x, f, t, x_sample): 
    # Here, take x_sample as a parameter for the ease of 
    # using both the F and F_fixed functions in this framework

    sum_R = 0

    for i in range(t):

        x_sample = np.random.uniform(0,1, x.shape)

        R_t = x >= x_sample

        sum_R = sum_R + f(R_t)

    return sum_R/t

# In Function F, re-sample from x every time we run the function
print(F_unfixed(x, f_linear, 100, x_sample))
print(F_unfixed(x, f_linear, 100, x_sample))
print(F_unfixed(x, f_linear, 100, x_sample))

-0.7980966155975172
-0.99250439092805
-0.9334419022178256


In [50]:
# Sample from x (stored in x_sample) once and for all  
# x_sample defined here will be used as a parameter for the code below

def get_x_sample(t): 
    x_sample = []
    for i in range(t):
        x_sample.append(np.random.uniform(0,1, x.shape))
        
    return x_sample

In [51]:
x_sample = get_x_sample(t)

def F_fixed(x, f, t, x_sample):
    # Here, we do not re-sample from x every time we run the function
    # Instead, we take a collection of x samples and compute the estimated expectation
    
    sum_R = 0
    for i in range(t):
        R_t = x >= x_sample[i]
        sum_R = sum_R + f(R_t)
    return sum_R/t
                 
print(F_fixed(x, f_linear, 100, x_sample))
print(F_fixed(x, f_linear, 100, x_sample))
print(F_fixed(x, f_linear, 100, x_sample))

-1.0633340141700303
-1.0633340141700303
-1.0633340141700303


## Function to calculate gradients

In [81]:
def get_gradient_F(F,x,f,t,n,x_sample):
    # A vectorized function to get gradients
    
    x_new_1 = x*np.ones((n, n))
    x_new_0 = x*np.ones((n, n))

    np.fill_diagonal(x_new_1, 1)
    np.fill_diagonal(x_new_0, 0)
    
    return F(x_new_1, f, t, x_sample) - F(x_new_0, f, t, x_sample)

print(get_gradient_F(F_fixed,x,f_linear,t,n,x_sample))

[-0.73569541  0.62903284  0.26188165  0.63377371  0.44822793  0.70714339
 -0.16722055 -0.33568168 -0.29152329  0.56592234]


## Gradient ascent

In [230]:
def gradient_ascent(F, x, f, alpha, t, epsilon, n, n_step_max, grad_decay = False, input_x_sample = None):  
    
    if input_x_sample == None:
        x_sample = get_x_sample(t)
    else:
        x_sample = input_x_sample
    
    x = copy.deepcopy(x)
    x_init = copy.deepcopy(x)
    sum_init = F(x, f, t, x_sample)
    
    # key values to be used
    sum_update = 0
    step = 0
    
    sum_temp = copy.deepcopy(sum_init)
    
    # start updating the parameters x with iterative gradients
#     while (np.abs(sum_temp - sum_update) > epsilon) & (step < n_step_max) :    
    while  (step < n_step_max) :  
#     while (np.abs(sum_temp - sum_update) > epsilon) & (sum_update > sum_temp) & (step < n_step_max) :
  
        step += 1
        sum_temp = F(x, f, t, x_sample)
        x_temp = x
        
        grad = get_gradient_F(F,x,f,t,n, x_sample)
        
        x = x + alpha*grad
        x = np.maximum(np.minimum(x,1),0)
        
        sum_update = F(x, f, t, x_sample)
        
        if grad_decay == True:
            alpha *= (1/step)**(1/5)
            print("alpha: ", alpha, '\n')
        
    print("\n",
          "Iteration: ", step, "\n\n" , 
          "Gradient:  ", grad, "\n\n",
          "x updated: ", x, "\n",
          "Updated  value: ", sum_update,"\n\n", 
          "x second last: ", x_temp, "\n",
          "Second last value: ", sum_temp, "\n\n")
    
        
    
    return step,sum_update, x, x_temp, x_sample

# Testing with a linear function

In [231]:
###
# parameters
###

# coefficients for the linear function
a = np.random.uniform(-1,1, n)

# parameters fo the gradient ascent function
alpha = 0.01
epsilon = 10**(-5000)
n_step_max = 5000

# number of iterations for estimating the expectation
t = 300

# number of products in the assortment
n = 10

# initialize the x vector
x_initial = np.random.uniform(0,1, n)
print(x_initial)

[0.00337801 0.43522002 0.06116302 0.9202457  0.7709123  0.76154319
 0.29273609 0.57126824 0.98487723 0.09116732]


### Testing for different values of a

In [232]:
# Use function F_fixed: sampling for estimating expectation is only done once and for all
print(a)
# x_initial = np.random.uniform(0,1,n)
step_,sum_update_, x_, x_temp_, x_sample_ = gradient_ascent(F_fixed, x_initial, f_linear, alpha, t, epsilon, n, 
                                                            n_step_max, grad_decay=False, input_x_sample = x_sample)

[ 0.11554293  0.21332587  0.07376636 -0.71596824 -0.6905506   0.25964938
  0.35542768 -0.93685592  0.12339781 -0.1078814 ]

 Iteration:  5000 

 Gradient:   [ 0.11554293  0.21332587  0.07376636 -0.71596824 -0.6905506   0.25964938
  0.35542768 -0.93685592  0.12339781 -0.1078814 ] 

 x updated:  [1. 1. 1. 0. 0. 1. 1. 0. 1. 0.] 
 Updated  value:  1.1411100383653545 

 x second last:  [1. 1. 1. 0. 0. 1. 1. 0. 1. 0.] 
 Second last value:  1.1411100383653545 




#### Check that the last two iterations actually converged

In [214]:
x_previous = x_temp_
print("x (2nd last iter): ", x_previous)
print("Value of F (2nd last iter): ", F_fixed(x_previous, f_linear, t, x_sample_), "\n")

x_updated  = x_
print("x (last iter): ", x_updated)
print("Value of F (last iter): ", F_fixed(x_updated, f_linear, t, x_sample_))

x (2nd last iter):  [0.1256412  1.         0.63814269 1.         0.38902341 0.21389091
 1.         0.         1.         1.        ]
Value of F (2nd last iter):  2.551763805394968 

x (last iter):  [0.12048984 1.         0.6354695  1.         0.38639506 0.21471746
 1.         0.         1.         1.        ]
Value of F (last iter):  2.551763805394968


#### If we manually converge probs to 0 or 1...

In [215]:
x_previous_modified = copy.deepcopy(x_previous)
x_previous_modified[0] = 0
x_previous_modified[6] = 1

print(x_previous_modified)
F_fixed(x_previous_modified, f_linear, t, x_sample_) 

[0.         1.         0.63814269 1.         0.38902341 0.21389091
 1.         0.         1.         1.        ]


2.601560301488282

#### The convergence will be actually much faster if we use decaying gradient here
Due to the fact that there is no local maximum in the linear function.
And that decreasing in steps will make the incremental change really small.

In [216]:
step_,sum_update_, x_, x_temp_, x_sample_  = gradient_ascent(F_fixed, x, f_linear, 
                                                             alpha, t, epsilon, n, n_step_max, 
                                                             grad_decay=True, input_x_sample=x_sample)

alpha:  0.01 

alpha:  0.008705505632961241 

alpha:  0.0069882711877157925 

alpha:  0.005296119205244061 

alpha:  0.0038385194963737744 

alpha:  0.0026824615199994182 

alpha:  0.0018176652007284484 

alpha:  0.0011992118057488942 

alpha:  0.000772764910314653 

alpha:  0.0004875816957196081 

alpha:  0.000301834484571944 

alpha:  0.00018362533756728568 

alpha:  0.00010993701395130539 


 Iteration:  13 

 Gradient:   [-0.51513617  0.85715328 -0.2673194   0.77575729 -0.26283486  0.08265449
  0.60366306 -0.94742592  0.52404917  0.09832171] 

 x updated:  [0.17152594 0.04480645 0.48227552 0.72706698 0.06837715 1.
 0.41338185 0.95047467 0.02739392 0.93411723] 
 Updated  value:  -0.07578491974910434 

 x second last:  [0.17162053 0.04464906 0.48232461 0.72692453 0.06842541 1.
 0.413271   0.95064865 0.02729769 0.93409917] 
 Second last value:  -0.07578491974910434 




#### Initialization on x matters

In [221]:
x_initial = np.random.uniform(0,1,n)
step_,sum_update_, x_, x_temp_, x_sample_ = gradient_ascent(F_fixed, x_initial, f_linear, alpha, t, epsilon, n, 
                                                            n_step_max,grad_decay=False, input_x_sample = x_sample)


 Iteration:  131 

 Gradient:   [-0.51513617  0.85715328 -0.2673194   0.77575729 -0.26283486  0.08265449
  0.60366306 -0.94742592  0.52404917  0.09832171] 

 x updated:  [0.         1.         0.34993259 1.         0.         1.
 1.         0.         1.         0.18470637] 
 Updated  value:  2.763326688867575 

 x second last:  [0.         1.         0.35260578 1.         0.         1.
 1.         0.         1.         0.18372315] 
 Second last value:  2.763326688867575 




In [222]:
x_initial = np.random.uniform(0,1,n)
step_,sum_update_, x_, x_temp_, x_sample_ = gradient_ascent(F_fixed, x_initial, f_linear, alpha, t, epsilon, n, 
                                                            n_step_max,grad_decay=False, input_x_sample = x_sample)


 Iteration:  157 

 Gradient:   [-0.51513617  0.85715328 -0.2673194   0.77575729 -0.26283486  0.08265449
  0.60366306 -0.94742592  0.52404917  0.09832171] 

 x updated:  [0.         1.         0.28371171 1.         0.         0.31779926
 1.         0.         1.         0.47334486] 
 Updated  value:  2.7536242510370426 

 x second last:  [0.         1.         0.28638491 1.         0.         0.31697271
 1.         0.         1.         0.47236164] 
 Second last value:  2.7536242510370426 




In [223]:
# Use function F: re-sampling is enabled in each iteration

# Note: in this case, the value of the function could decrease as each time the estimated value of F is different 
#       due to differnt samples from x
step_,sum_update_, x_, x_temp_, x_sample_  = gradient_ascent(F_unfixed, x_initial, f_linear, alpha, t, epsilon,
                                                             n, n_step_max, grad_decay=False)


 Iteration:  535 

 Gradient:   [-0.51906903  0.85084523 -0.26919578  0.77708264 -0.26767529  0.0852764
  0.60822797 -0.94978136  0.51667688  0.09556656] 

 x updated:  [0.         1.         0.         1.         0.         0.63707937
 1.         0.         1.         0.84470702] 
 Updated  value:  2.898524920965915 

 x second last:  [0.         1.         0.         1.         0.         0.63622661
 1.         0.         1.         0.84375136] 
 Second last value:  2.8985249209659183 


